<a href="https://colab.research.google.com/github/FIGARO79/GanaBaloto/blob/main/GanaBalotoColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

if not os.path.exists('/content/drive/MyDrive/GanaBaloto'):
  print("Installing and Running For First Time.")
  %cd /content/drive/MyDrive
  !git clone https://github.com/FIGARO79/GanaBaloto.git
  !cp -r /content/drive/MyDrive/GanaBaloto
else:
  print("Repository already exist. Continue...")
  %cd /content/drive/MyDrive/GanaBaloto

import pandas as pd
from IPython.display import display, HTML
import random
from itertools import combinations
import numpy as np


# Update the file path to include the full path from your Google Drive
file_path = '/content/drive/MyDrive/GanaBaloto/resultados_aciertos.xlsx'
sheet_data = pd.read_excel(file_path, sheet_name='data')
sheet_data = sheet_data.dropna(subset=['SB'])
sheet_data['SB'] = sheet_data['SB'].astype(int)

# Calculate frequencies for all draws
all_draws_frequencies = {}
columns_to_analyze = ['B1', 'B2', 'B3', 'B4', 'B5']
for column in columns_to_analyze:
    all_draws_frequencies[column] = sheet_data[column].value_counts().head(5).index.tolist()
all_draws_frequencies["Super Balota"] = sheet_data["SB"].value_counts().head(5).index.tolist()

# Filter data for draws with 'Premios 5+1' > 0
filtered_data = sheet_data[sheet_data['Premios 5+1'] > 0]

# Calculate frequencies for filtered draws
refined_predictions = {}
for column in columns_to_analyze:
    refined_predictions[column] = filtered_data[column].value_counts().head(5).index.tolist()
refined_predictions["Super Balota"] = filtered_data['SB'].value_counts().head(5).index.tolist()

df_all_draws = pd.DataFrame(all_draws_frequencies).T.rename(columns={i: f'Top {i+1}' for i in range(5)})
df_refined_predictions = pd.DataFrame(refined_predictions).T.rename(columns={i: f'Top {i+1}' for i in range(5)})


# Combination Generation

def calculate_probability_numpy(combination, sb, frequencies_df):
    """Calculates the frequency score for a given combination using NumPy."""
    total_draws = frequencies_df.shape[0]
    probability = 0

    for i, number in enumerate(combination):
        column_name = f'B{i+1}'
        # Use NumPy to calculate frequency and probability
        column_data = frequencies_df[column_name].to_numpy()
        frequency = np.sum(column_data == number)
        probability += frequency / total_draws

    # Use NumPy for Super Balota calculation
    sb_data = frequencies_df['SB'].to_numpy()
    sb_frequency = np.sum(sb_data == sb)
    probability += sb_frequency / total_draws

    return probability # Changed this line to return the calculated probability

def generate_probable_combinations(predictions, num_combinations=10, frequencies_df=sheet_data):  # Add frequencies_df as argument
    """
    Generates combinations based on the most frequent numbers and calculates their probabilities.

    Args:
        predictions (dict): Dictionary containing the most frequent numbers for each column.
        num_combinations (int): Number of combinations to generate.
        frequencies_df (pd.DataFrame): DataFrame containing the frequency data.

    Returns:
        list: A list of probable combinations with their probabilities.
    """
    all_probable_numbers = [number for sublist in predictions.values() for number in sublist]
    # Ensure unique numbers while preserving order (using dict.fromkeys)
    unique_probable_numbers = list(dict.fromkeys(all_probable_numbers))

    probable_combinations = []
    for _ in range(num_combinations):
        combination = random.sample(unique_probable_numbers, 5)  # Pick 5 unique numbers
        sb = random.choice(predictions["Super Balota"])  # Pick Super Balota from its predictions

        probability = calculate_probability_numpy(combinación, sb, frequencies_df) # Calculate probability
        probable_combinations.append((combination, sb, probability))  # Store with probability

    return probable_combinations

# Generate and print 10 probable combinations with probabilities
probable_combinations = generate_probable_combinations(refined_predictions, num_combinations=10)  # Pass sheet_data

# Create a list to store the data for the DataFrame
data = []
for combination, sb, probability in probable_combinations:
    data.append([', '.join(map(str, combination)), sb, probability])

df_combinations = pd.DataFrame(data, columns=['Combinación', 'Super Balota', 'Probability'], index=range(1, 11))
df_combinations['Probability'] = df_combinations['Probability'].map('{:.2%}'.format)

#print(f"\nCombinaciones con Probabilidadesde acierto:")
#print(df_combinations)
#display(HTML(df_combinations.to_html(escape=False)))

# Display tables side-by-side using HTML
html_str = f"""
<table><tr>
<td>
<h3>Números más frecuentes (todos los sorteos)</h3>
{df_all_draws.to_html()}
</td>
<td>
<h3>Números más frecuentes (solo sorteos con premio 5+1)</h3>
{df_refined_predictions.to_html()}
</td>
</tr></table>
"""
display(HTML(html_str))

html_str = f"""
<table><tr>
<td>
<h3>Combinaciones con Probabilidades de acierto</h3>
{df_combinations.to_html()}
</td>
</tr></table>
"""
display(HTML(html_str))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Repository already exist. Continue...
/content/drive/MyDrive/GanaBaloto


,Top 1,Top 2,Top 3,Top 4,Top 5
B1,1,2,3,6,5
B2,14,11,8,12,20
B3,25,20,22,24,28
B4,34,35,37,30,27
B5,43,42,41,40,39
Super Balota,11,7,13,16,3
,Top 1,Top 2,Top 3,Top 4,Top 5
B1,8,5,9,2,6
B2,6,9,16,11,17
B3,19,13,14,28,34


,Combination,Super Balota,Probability
1,"15, 8, 19, 43, 17",14,17.23%
2,"9, 26, 28, 19, 31",14,21.47%
3,"15, 11, 13, 23, 14",13,21.22%
4,"13, 7, 41, 35, 16",13,19.48%
5,"43, 5, 35, 26, 28",14,15.61%
6,"9, 26, 34, 19, 5",15,14.61%
7,"17, 8, 19, 35, 16",7,24.34%
8,"40, 6, 34, 8, 5",7,12.23%
9,"9, 2, 31, 26, 11",14,18.48%
10,"8, 13, 26, 18, 23",15,20.72%
